In [1]:
ls

Find epitopes on healthy tissue that could cross-react with MAGE.ipynb
hla_2020.12/


In [16]:
from pathlib import Path
from typing import Optional, Sequence, Tuple, Dict

import numpy as np
import pandas as pd

try:
    from tqdm.auto import tqdm
except ImportError:
    def tqdm(x, **kwargs):  # fallback
        return x


# -----------------------------
# 1) Simple TCR-ish substitution (20x20)
# -----------------------------
AA_LIST = "ACDEFGHIKLMNPQRSTVWY"
AA_TO_IDX = {aa: i for i, aa in enumerate(AA_LIST)}

HYDROPHOBIC = set("AVLIM")
AROMATIC    = set("FYW")
POLAR       = set("STNQC")
NEGATIVE    = set("DE")
POSITIVE    = set("KRH")
SPECIAL     = set("GP")

CHEAP_PAIRS = {
    frozenset(("I","L")), frozenset(("I","V")), frozenset(("L","V")),
    frozenset(("D","E")),
    frozenset(("K","R")),
    frozenset(("N","Q")), frozenset(("S","T")),
    frozenset(("F","Y")),
}

def sub_cost_simple_tcr(a: str, b: str) -> float:
    a = a.upper(); b = b.upper()
    if a == b:
        return 0.0
    if a not in AA_TO_IDX or b not in AA_TO_IDX:
        return 1.0

    if frozenset((a, b)) in CHEAP_PAIRS:
        base = 0.05
    elif (a in NEGATIVE and b in POSITIVE) or (a in POSITIVE and b in NEGATIVE):
        base = 1.00
    elif (a in NEGATIVE and b in NEGATIVE) or (a in POSITIVE and b in POSITIVE):
        base = 0.15
    elif (a in HYDROPHOBIC and b in HYDROPHOBIC) or (a in AROMATIC and b in AROMATIC):
        base = 0.20
    elif (a in POLAR and b in POLAR):
        base = 0.20
    elif (a in HYDROPHOBIC and b in AROMATIC) or (a in AROMATIC and b in HYDROPHOBIC):
        base = 0.35
    elif ((a in POLAR) and (b in NEGATIVE or b in POSITIVE)) or ((b in POLAR) and (a in NEGATIVE or a in POSITIVE)):
        base = 0.80
    elif ((a in HYDROPHOBIC or a in AROMATIC) and (b in NEGATIVE or b in POSITIVE)) or ((b in HYDROPHOBIC or b in AROMATIC) and (a in NEGATIVE or a in POSITIVE)):
        base = 0.95
    else:
        base = 0.70

    if a in SPECIAL or b in SPECIAL:
        base = min(1.0, base + 0.15)
    return float(base)

def build_sub_matrix() -> np.ndarray:
    sub = np.ones((20, 20), dtype=np.float32)
    for i, a in enumerate(AA_LIST):
        for j, b in enumerate(AA_LIST):
            sub[i, j] = np.float32(sub_cost_simple_tcr(a, b))
    return sub

SUB_MAT = build_sub_matrix()


# -----------------------------
# 2) Levenshtein DP: unit + weighted (indel=1.0 per residue)
# -----------------------------
def encode(seq: str) -> np.ndarray:
    return np.array([AA_TO_IDX.get(ch, -1) for ch in seq.upper()], dtype=np.int16)

def lev_unit_and_weighted(p_idx: np.ndarray, t_idx: np.ndarray, sub_mat: np.ndarray, indel: float = 1.0) -> Tuple[int, float]:
    m = int(p_idx.shape[0]); n = int(t_idx.shape[0])

    prev_u = np.arange(n + 1, dtype=np.int16)
    prev_w = (np.arange(n + 1, dtype=np.float32) * np.float32(indel))

    for i in range(m):
        cur_u = np.empty(n + 1, dtype=np.int16)
        cur_w = np.empty(n + 1, dtype=np.float32)
        cur_u[0] = i + 1
        cur_w[0] = np.float32((i + 1) * indel)

        pi = int(p_idx[i])

        for j in range(1, n + 1):
            tj = int(t_idx[j - 1])

            # unit
            mismatch = 1
            if pi != -1 and tj != -1 and pi == tj:
                mismatch = 0
            sub_u = prev_u[j - 1] + mismatch
            del_u = prev_u[j] + 1
            ins_u = cur_u[j - 1] + 1
            cur_u[j] = min(sub_u, del_u, ins_u)

            # weighted (sub <= 1, indel = 1)
            sc = 1.0 if (pi == -1 or tj == -1) else float(sub_mat[pi, tj])
            sub_w = prev_w[j - 1] + sc
            del_w = prev_w[j] + indel
            ins_w = cur_w[j - 1] + indel
            cur_w[j] = min(sub_w, del_w, ins_w)

        prev_u, prev_w = cur_u, cur_w

    return int(prev_u[n]), float(prev_w[n])


# -----------------------------
# 3) Atlas load + filters
# -----------------------------
def normalize_hla_class(x: pd.Series) -> pd.Series:
    s = x.astype(str).str.strip().str.upper()
    s = s.str.replace(r"\s+", "", regex=True)
    s = s.replace({"I": "HLA-I", "II": "HLA-II"})
    s = s.str.replace("HLAI", "HLA-I", regex=False).str.replace("HLAII", "HLA-II", regex=False)
    return s

def normalize_allele(a: Optional[str]) -> Optional[str]:
    if a is None:
        return None
    a = a.strip()
    return a[4:] if a.upper().startswith("HLA-") else a

def load_atlas(data_dir: str | Path = "hla_2020.12") -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    d = Path(data_dir)
    pep_path = d / "HLA_peptides.tsv"
    hit_path = d / "HLA_sample_hits.tsv"
    don_path = d / "HLA_donors.tsv"
    for p in (pep_path, hit_path, don_path):
        if not p.exists():
            raise FileNotFoundError(f"Missing: {p}")

    peptides = pd.read_csv(pep_path, sep="\t", usecols=["peptide_sequence_id", "peptide_sequence"])
    hits     = pd.read_csv(hit_path, sep="\t", usecols=["peptide_sequence_id", "donor", "tissue", "hla_class"])
    donors   = pd.read_csv(don_path, sep="\t", usecols=["donor", "hla_allele"])

    peptides["peptide_sequence_id"] = peptides["peptide_sequence_id"].astype(str).str.strip()
    peptides["peptide_sequence"]    = peptides["peptide_sequence"].astype(str).str.strip().str.upper()

    hits["peptide_sequence_id"] = hits["peptide_sequence_id"].astype(str).str.strip()
    hits["donor"] = hits["donor"].astype(str).str.strip()
    hits["tissue"] = hits["tissue"].astype(str).str.strip()
    hits["hla_class_norm"] = normalize_hla_class(hits["hla_class"])

    donors["donor"] = donors["donor"].astype(str).str.strip()
    donors["hla_allele"] = donors["hla_allele"].astype(str).str.strip().map(normalize_allele)

    print("\nLoaded:")
    print("  peptides:", peptides.shape, "| unique IDs:", peptides["peptide_sequence_id"].nunique())
    print("  hits:    ", hits.shape, "| unique IDs:", hits["peptide_sequence_id"].nunique())
    print("  donors:  ", donors.shape, "| unique donors:", donors["donor"].nunique(), "| unique alleles:", donors["hla_allele"].nunique())
    print("\nHits class counts:")
    print(hits["hla_class_norm"].value_counts().head(10))
    return peptides, hits, donors


# -----------------------------
# 4) End-prefilter (parametric) to bound candidates
# -----------------------------
def ends_prefilter_mask(
    epitope: str,
    seqs: Sequence[str],
    *,
    prefix_len: int = 3,
    suffix_len: int = 3,
    prefix_min_frac: float = 1/3,
    suffix_min_frac: float = 1/3,
    enabled: bool = True,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Keep seq if:
      matches(ep[:prefix_len], seq[:prefix_len]) >= ceil(effective_prefix_len * prefix_min_frac)
   AND
      matches(ep[-suffix_len:], seq[-suffix_len:]) >= ceil(effective_suffix_len * suffix_min_frac)

    Returns: (mask, score) where score = prefix_matches + suffix_matches (for ranking/capping).
    """
    if not enabled:
        mask = np.ones(len(seqs), dtype=bool)
        score = np.zeros(len(seqs), dtype=np.int16)
        return mask, score

    ep = epitope.upper()
    m = len(ep)
    mask = np.zeros(len(seqs), dtype=bool)
    score = np.zeros(len(seqs), dtype=np.int16)

    for i, s in enumerate(seqs):
        s = s.upper()
        L = len(s)

        preL = min(prefix_len, m, L)
        sufL = min(suffix_len, m, L)

        # dynamic minima based on effective compared length
        pre_min = int(np.ceil(preL * prefix_min_frac)) if preL > 0 else 0
        suf_min = int(np.ceil(sufL * suffix_min_frac)) if sufL > 0 else 0

        pre = 0
        for k in range(preL):
            if ep[k] == s[k]:
                pre += 1

        suf = 0
        for k in range(sufL):
            if ep[-(k+1)] == s[-(k+1)]:
                suf += 1

        score[i] = pre + suf
        mask[i] = (pre >= pre_min) and (suf >= suf_min)

    return mask, score


# -----------------------------
# 5) Tissue grouping + summaries
# -----------------------------
def norm_tissue(s: str) -> str:
    return " ".join(str(s).strip().lower().replace("_", " ").split())

def join_unique(vals: pd.Series) -> str:
    u = sorted(set(v for v in vals.dropna().astype(str).tolist() if v and v != "nan"))
    return ",".join(u)

def add_tissue_groups(df: pd.DataFrame, protected: Sequence[str], tolerated: Sequence[str]) -> pd.DataFrame:
    out = df.copy()
    prot = {norm_tissue(x) for x in protected}
    tol  = {norm_tissue(x) for x in tolerated}
    out["tissue_norm"] = out["tissue"].map(norm_tissue)
    out["tissue_group"] = np.where(
        out["tissue_norm"].isin(prot),
        "protected",
        np.where(out["tissue_norm"].isin(tol), "tolerated", "uncertain")
    )
    out["one"] = 1
    return out


# -----------------------------
# 6) Run one job
# -----------------------------
def run_epitope(
    peptides: pd.DataFrame,
    hits: pd.DataFrame,
    donors: pd.DataFrame,
    *,
    epitope: str,
    allele: Optional[str],
    job: str,
    out_dir: str | Path = "out_neighbors",
    max_unit: int = 4,
    max_weighted: float = 4.0,
    protected_tissues: Sequence[str] = (),
    tolerated_tissues: Sequence[str] = (),
    # end-prefilter params
    end_prefilter_enabled: bool = True,
    prefix_len: int = 3,
    suffix_len: int = 3,
    prefix_min_frac: float = 1/3,
    suffix_min_frac: float = 1/3,
    # bound candidates (optional)
    max_candidates: Optional[int] = None,
) -> Dict[str, pd.DataFrame]:

    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)

    ep = epitope.strip().upper()
    allele = normalize_allele(allele)
    print("\n" + "="*90)
    print(f"{job}: epitope={ep} | allele(donor filter)={allele or 'None'}")
    print(f"  thresholds: unit<={max_unit}, weighted<={max_weighted}")
    print(f"  end-prefilter: {end_prefilter_enabled} | prefix_len={prefix_len}, suffix_len={suffix_len}, "
          f"prefix_min_frac={prefix_min_frac}, suffix_min_frac={suffix_min_frac}")
    print("="*90)

    # 1) HLA-I only
    h = hits.loc[hits["hla_class_norm"] == "HLA-I", ["peptide_sequence_id", "donor", "tissue"]].copy()

    # 2) donor allele filter (THIS happens before collecting obs_ids and before distances)
    if allele is not None:
        donors_with = set(donors.loc[donors["hla_allele"] == allele, "donor"].tolist())
        h = h.loc[h["donor"].isin(donors_with)].copy()
        print(f"  donors carrying {allele}: {len(donors_with)} | HLA-I hits after donor filter: {len(h)}")

    if h.empty:
        print("  No hits after filtering; skipping.")
        return {}

    # 3) restrict candidate peptides to observed IDs (after allele filter)
    obs_ids = set(h["peptide_sequence_id"].unique().tolist())
    p = peptides.loc[peptides["peptide_sequence_id"].isin(obs_ids)].copy()

    # 4) length window (necessary lower bound for edit distance <= K because indel=1)
    max_len_diff = int(max(max_unit, np.floor(max_weighted)))
    m = len(ep)
    p["pep_len"] = p["peptide_sequence"].str.len().astype(np.int16)
    p = p.loc[(p["pep_len"] >= max(1, m - max_len_diff)) & (p["pep_len"] <= m + max_len_diff)].copy()
    print(f"  candidates after observed-ID + length window: {len(p)}")

    if p.empty:
        print("  No candidate peptides after length window; skipping.")
        return {}

    # 5) end-prefilter to shrink candidate set before DP
    seqs = p["peptide_sequence"].tolist()
    mask, score = ends_prefilter_mask(
        ep, seqs,
        enabled=end_prefilter_enabled,
        prefix_len=prefix_len, suffix_len=suffix_len,
        prefix_min_frac=prefix_min_frac, suffix_min_frac=suffix_min_frac
    )
    p["endmatch_score"] = score
    before = len(p)
    p = p.loc[mask].copy()
    print(f"  end-prefilter kept {len(p)} / {before} candidates")

    # optional hard cap: keep best endmatch_score (then closest length)
    if max_candidates is not None and len(p) > max_candidates:
        p = p.sort_values(["endmatch_score", "pep_len"], ascending=[False, True]).head(max_candidates).copy()
        print(f"  capped to max_candidates={max_candidates}: now {len(p)}")

    if p.empty:
        print("  No candidates after end-prefilter/cap; skipping.")
        return {}

    # 6) compute unit + weighted distances
    p_idx = encode(ep)
    unit = np.empty(len(p), dtype=np.int16)
    wlev = np.empty(len(p), dtype=np.float32)

    for i, s in enumerate(tqdm(p["peptide_sequence"].tolist(), desc=f"{job}: DP", leave=False)):
        u, w = lev_unit_and_weighted(p_idx, encode(s), SUB_MAT, indel=1.0)
        unit[i] = u
        wlev[i] = w

    p["unit_dist"] = unit
    p["weighted_dist"] = wlev

    p["pass_unit"] = p["unit_dist"] <= max_unit
    p["pass_weighted"] = p["weighted_dist"] <= float(max_weighted)
    p["pass_union"] = p["pass_unit"] | p["pass_weighted"]

    # helper: build outputs for one label
    def make_block(label: str, mask_col: str) -> Tuple[pd.DataFrame, pd.DataFrame]:
        neigh = p.loc[p[mask_col], ["peptide_sequence_id", "peptide_sequence", "pep_len", "unit_dist", "weighted_dist"]].copy()
        neigh = neigh.rename(columns={"peptide_sequence": "peptide"})
        hj = h.merge(neigh, on="peptide_sequence_id", how="inner")
        hj = add_tissue_groups(hj, protected_tissues, tolerated_tissues)

        overall = (hj.groupby("peptide", as_index=False)
                   .agg(
                       pep_len=("pep_len", "first"),
                       unit_dist=("unit_dist", "first"),
                       weighted_dist=("weighted_dist", "first"),
                       n_hits=("one", "sum"),
                       n_donors=("donor", "nunique"),
                       n_tissues=("tissue", "nunique"),
                       tissues=("tissue", join_unique),
                   ))

        hit_counts = (hj.pivot_table(index="peptide", columns="tissue_group", values="one",
                                     aggfunc="sum", fill_value=0)
                      .rename(columns={"protected": "protected_hits", "tolerated": "tolerated_hits", "uncertain": "uncertain_hits"})
                      .reset_index())

        tis_counts = (hj.pivot_table(index="peptide", columns="tissue_group", values="tissue",
                                     aggfunc=pd.Series.nunique, fill_value=0)
                      .rename(columns={"protected": "protected_tissues_n", "tolerated": "tolerated_tissues_n", "uncertain": "uncertain_tissues_n"})
                      .reset_index())

        tis_lists = (hj.groupby(["peptide", "tissue_group"])["tissue"]
                     .apply(join_unique)
                     .unstack(fill_value="")
                     .rename(columns={"protected": "protected_tissues", "tolerated": "tolerated_tissues", "uncertain": "uncertain_tissues"})
                     .reset_index())

        summary = (overall.merge(hit_counts, on="peptide", how="left")
                          .merge(tis_counts, on="peptide", how="left")
                          .merge(tis_lists, on="peptide", how="left"))

        # ensure group cols exist
        for c in ["protected_hits","tolerated_hits","uncertain_hits",
                  "protected_tissues_n","tolerated_tissues_n","uncertain_tissues_n",
                  "protected_tissues","tolerated_tissues","uncertain_tissues"]:
            if c not in summary.columns:
                summary[c] = 0 if c.endswith(("_hits","_n")) else ""

        summary = summary.sort_values(
            ["unit_dist", "weighted_dist", "protected_hits", "n_tissues", "n_donors", "peptide"],
            ascending=[True, True, False, False, False, True]
        ).reset_index(drop=True)

        return hj, summary

    blocks = {}
    for label, mask in [("union", "pass_union"), ("unit", "pass_unit"), ("weighted", "pass_weighted")]:
        hj, summ = make_block(label, mask)

        # provenance columns (for later combining)
        hj.insert(0, "label", label)
        hj.insert(0, "query_allele", allele or "")
        hj.insert(0, "query_epitope", ep)
        hj.insert(0, "job", job)

        summ.insert(0, "label", label)
        summ.insert(0, "query_allele", allele or "")
        summ.insert(0, "query_epitope", ep)
        summ.insert(0, "job", job)

        blocks[f"hits_{label}"] = hj
        blocks[f"summary_{label}"] = summ

        hj.to_csv(out_dir / f"{job}_{label}_all_hits.tsv", sep="\t", index=False)
        summ.to_csv(out_dir / f"{job}_{label}_summary.tsv", sep="\t", index=False)

        print(f"  wrote {label}: peptides={summ.shape[0]} | hits={hj.shape[0]}")

    return blocks


# -----------------------------
# 7) Run all jobs + write combined TSVs
# -----------------------------
def run_jobs(
    jobs: Sequence[Tuple[str, Optional[str], str]],
    *,
    data_dir: str | Path = "hla_2020.12",
    out_dir: str | Path = "out_neighbors",
    max_unit: int = 4,
    max_weighted: float = 4.0,
    protected_tissues: Sequence[str] = (),
    tolerated_tissues: Sequence[str] = (),
    # end-prefilter params
    end_prefilter_enabled: bool = True,
    prefix_len: int = 3,
    suffix_len: int = 3,
    prefix_min_frac: float = 1/3,
    suffix_min_frac: float = 1/3,
    max_candidates: Optional[int] = None,
) -> Dict[str, Dict[str, pd.DataFrame]]:

    out_dir = Path(out_dir); out_dir.mkdir(parents=True, exist_ok=True)
    peptides, hits, donors = load_atlas(data_dir)

    all_hit_rows = []
    all_sum_rows = []

    results = {}
    for ep, al, job in tqdm(list(jobs), desc="Epitopes", leave=True):
        res = run_epitope(
            peptides, hits, donors,
            epitope=ep,
            allele=al,
            job=job,
            out_dir=out_dir,
            max_unit=max_unit,
            max_weighted=max_weighted,
            protected_tissues=protected_tissues,
            tolerated_tissues=tolerated_tissues,
            end_prefilter_enabled=end_prefilter_enabled,
            prefix_len=prefix_len, suffix_len=suffix_len,
            prefix_min_frac=prefix_min_frac, suffix_min_frac=suffix_min_frac,
            max_candidates=max_candidates,
        )
        results[job] = res

        for k, df in res.items():
            if k.startswith("hits_"):
                all_hit_rows.append(df)
            elif k.startswith("summary_"):
                all_sum_rows.append(df)

    # Combined TSVs (all labels + all jobs)
    if all_hit_rows:
        hits_all = pd.concat(all_hit_rows, ignore_index=True)
        hits_all.to_csv(out_dir / "ALL_JOBS_all_hits.tsv", sep="\t", index=False)
        print("\nWrote combined hits:", out_dir / "ALL_JOBS_all_hits.tsv")

    if all_sum_rows:
        summ_all = pd.concat(all_sum_rows, ignore_index=True)
        summ_all.to_csv(out_dir / "ALL_JOBS_all_summaries.tsv", sep="\t", index=False)
        print("Wrote combined summaries:", out_dir / "ALL_JOBS_all_summaries.tsv")

    return results


# -----------------------------
# Defaults
# -----------------------------
PROTECTED = ["Brain", "Colon", "Heart", "Liver", "Kidney", "Cerebellum", "Small intestine", "Bone marrow", "Pancreas", "Aorta"]
TOLERATED = ["Ovary", "Uterus", "Testis", "Thymus", "Spleen"]

jobs = [
    ("SLLQHLIGL", "A*02:01", "PRAME_A02_SLLQHLIGL_on"),

    ("EVDPIGHLY", "A*01:01", "MAGEA3_A01_EVDPIGHLY_on"),
    ("ESDPIVAQY", "A*01:01", "TITIN_A01_ESDPIVAQY_off"),

    ("KVAELVHFL", "A*02:01", "MAGEA3_112_A02_KVAELVHFL_on"),
    ("KMAELVHFL", "A*02:01", "MAGEA12_A02_KMAELVHFL_off"),
]

# Run
results = run_jobs(
    jobs,
    data_dir="hla_2020.12",
    out_dir="out_neighbors",
    max_unit=4,
    max_weighted=4.0,

    protected_tissues=PROTECTED,
    tolerated_tissues=TOLERATED,

    # prefilter: ≥1/3 of first 3 AND ≥1/3 of last 3 match
    end_prefilter_enabled=True,
    prefix_len=3, suffix_len=3,
    prefix_min_frac=1/3, suffix_min_frac=1/3,

    # optional additional hard-bound, e.g. 50000 (set None to disable)
    max_candidates=None,
)



Loaded:
  peptides: (223246, 2) | unique IDs: 223246
  hits:     (967449, 5) | unique IDs: 223246
  donors:   (303, 2) | unique donors: 21 | unique alleles: 137

Hits class counts:
hla_class_norm
HLA-II    557946
HLA-I     409503
Name: count, dtype: int64


Epitopes:   0%|          | 0/5 [00:00<?, ?it/s]


PRAME_A02_SLLQHLIGL_on: epitope=SLLQHLIGL | allele(donor filter)=A*02:01
  thresholds: unit<=4, weighted<=4.0
  end-prefilter: True | prefix_len=3, suffix_len=3, prefix_min_frac=0.3333333333333333, suffix_min_frac=0.3333333333333333
  donors carrying A*02:01: 6 | HLA-I hits after donor filter: 141127
  candidates after observed-ID + length window: 40246
  end-prefilter kept 4470 / 40246 candidates


PRAME_A02_SLLQHLIGL_on: DP:   0%|          | 0/4470 [00:00<?, ?it/s]

  wrote union: peptides=1759 | hits=8059
  wrote unit: peptides=73 | hits=431
  wrote weighted: peptides=1759 | hits=8059

MAGEA3_A01_EVDPIGHLY_on: epitope=EVDPIGHLY | allele(donor filter)=A*01:01
  thresholds: unit<=4, weighted<=4.0
  end-prefilter: True | prefix_len=3, suffix_len=3, prefix_min_frac=0.3333333333333333, suffix_min_frac=0.3333333333333333
  donors carrying A*01:01: 7 | HLA-I hits after donor filter: 135954
  candidates after observed-ID + length window: 44459
  end-prefilter kept 2744 / 44459 candidates


...

  wrote union: peptides=810 | hits=6888
  wrote unit: peptides=20 | hits=202
  wrote weighted: peptides=810 | hits=6888

MAGEA3_112_A02_KVAELVHFL_on: epitope=KVAELVHFL | allele(donor filter)=A*02:01
  thresholds: unit<=4, weighted<=4.0
  end-prefilter: True | prefix_len=3, suffix_len=3, prefix_min_frac=0.3333333333333333, suffix_min_frac=0.3333333333333333
  donors carrying A*02:01: 6 | HLA-I hits after donor filter: 141127
  candidates after observed-ID + length window: 40246
  end-prefilter kept 2274 / 40246 candidates


MAGEA3_112_A02_KVAELVHFL_on: DP:   0%|          | 0/2274 [00:00<?, ?it/s]

  wrote union: peptides=856 | hits=3607
  wrote unit: peptides=29 | hits=157
  wrote weighted: peptides=856 | hits=3607

MAGEA12_A02_KMAELVHFL_off: epitope=KMAELVHFL | allele(donor filter)=A*02:01
  thresholds: unit<=4, weighted<=4.0
  end-prefilter: True | prefix_len=3, suffix_len=3, prefix_min_frac=0.3333333333333333, suffix_min_frac=0.3333333333333333
  donors carrying A*02:01: 6 | HLA-I hits after donor filter: 141127
  candidates after observed-ID + length window: 40246
  end-prefilter kept 1744 / 40246 candidates


MAGEA12_A02_KMAELVHFL_off: DP:   0%|          | 0/1744 [00:00<?, ?it/s]

  wrote union: peptides=712 | hits=3037
  wrote unit: peptides=17 | hits=93
  wrote weighted: peptides=712 | hits=3037

Wrote combined hits: out_neighbors/ALL_JOBS_all_hits.tsv
Wrote combined summaries: out_neighbors/ALL_JOBS_all_summaries.tsv


In [21]:
df_neighbors = pd.read_csv("out_neighbors/ALL_JOBS_all_summaries.tsv", sep="\t")

In [47]:
df_neighbors.query("(weighted_dist < 3.3) & (label == 'weighted') & (query_epitope == 'EVDPIGHLY')").peptide

3978    EVLDFITHLY
3979     YVDSEGHLY
3980    RVDPIGPLSY
3981     NVDPVQHTY
3982     EIDKNDHLY
           ...    
4266     EEEKLLKLF
4267     EEELLDKVY
4268      DVELLKLE
4269    DLDDAQRTLY
4270     FVEVGRVAY
Name: peptide, Length: 64, dtype: object

In [56]:
df_neighbors.query("(weighted_dist <= 2) & (label == 'weighted') & (query_epitope == 'KVAELVHFL')")

,job,query_epitope,query_allele,label,peptide,pep_len,unit_dist,weighted_dist,n_hits,n_donors,...,tissues,protected_hits,tolerated_hits,uncertain_hits,protected_tissues_n,tolerated_tissues_n,uncertain_tissues_n,protected_tissues,tolerated_tissues,uncertain_tissues
6868,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,KVADLVLML,9,3,1.35,1,1,...,Ovary,0,1,0,0,1,0,NaN,Ovary,NaN
6870,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,RLAEVVHEL,9,4,1.10,2,2,...,"Ovary,Thymus",0,2,0,0,2,0,NaN,"Ovary,Thymus",NaN
6871,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,SLAELVHAV,9,4,1.25,62,6,...,"Adrenal gland,Aorta,Bladder,Bone marrow,Brain,...",25,11,26,10,5,10,"Aorta,Bone marrow,Brain,Cerebellum,Colon,Heart...","Ovary,Spleen,Testis,Thymus,Uterus","Adrenal gland,Bladder,Esophagus,Lung,Lymph nod..."
6872,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,KLAEFIDFL,9,4,1.45,3,3,...,"Bone marrow,Ovary,Thymus",1,2,0,1,2,0,Bone marrow,"Ovary,Thymus",NaN
6873,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,KVLDFEHFL,9,4,1.55,24,6,...,"Adrenal gland,Aorta,Bone marrow,Cerebellum,Col...",10,5,9,5,4,7,"Aorta,Bone marrow,Cerebellum,Colon,Kidney","Ovary,Spleen,Thymus,Uterus","Adrenal gland,Esophagus,Lung,Lymph node,Thyroi..."
6874,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,KVVALVHAV,9,4,1.55,2,2,...,"Lung,Ovary",0,1,1,0,1,1,NaN,Ovary,Lung
6875,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,NVAELLHQV,9,4,1.60,11,2,...,"Aorta,Bone marrow,Colon,Esophagus,Kidney,Liver...",5,1,5,5,1,5,"Aorta,Bone marrow,Colon,Kidney,Liver",Spleen,"Esophagus,Lymph node,Thyroid,Tongue,Trachea"
6876,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,KVASLLHQV,9,4,1.60,1,1,...,Ovary,0,1,0,0,1,0,NaN,Ovary,NaN
6877,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,SIAEVVHQL,9,4,1.60,2,2,...,"Ovary,Thymus",0,2,0,0,2,0,NaN,"Ovary,Thymus",NaN
6878,MAGEA3_112_A02_KVAELVHFL_on,KVAELVHFL,A*02:01,weighted,LVADLVHTV,9,4,1.75,5,2,...,"Colon,Kidney,Liver,Lung,Lymph node",3,0,2,3,0,2,"Colon,Kidney,Liver",NaN,"Lung,Lymph node"


In [77]:
df_prame = df_neighbors.query("(weighted_dist < 3.3) & (unit_dist <= 4) & (label == 'weighted') & (query_epitope == 'SLLQHLIGL')")
len(df_prame)

62

In [80]:
df_mage_a3_kva = df_neighbors.query("(weighted_dist < 3.3) & (unit_dist <= 4) & (label == 'weighted') & (query_epitope == 'KVAELVHFL')")
len(df_mage_a3_kva)

27

In [92]:
df_mage_evd = df_neighbors.query("(weighted_dist < 3.3) & (unit_dist <= 4)  & (label == 'weighted') & (query_epitope == 'EVDPIGHLY')")
len(df_mage_evd)

12

In [93]:
df_mage_evd

,job,query_epitope,query_allele,label,peptide,pep_len,unit_dist,weighted_dist,n_hits,n_donors,...,tissues,protected_hits,tolerated_hits,uncertain_hits,protected_tissues_n,tolerated_tissues_n,uncertain_tissues_n,protected_tissues,tolerated_tissues,uncertain_tissues
3978,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,EVLDFITHLY,10,3,2.70,1,1,...,Thymus,0,1,0,0,1,0,NaN,Thymus,NaN
3979,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,YVDSEGHLY,9,3,2.75,13,4,...,"Aorta,Bladder,Colon,Esophagus,Heart,Lung,Testi...",3,3,7,3,2,6,"Aorta,Colon,Heart","Testis,Thymus","Bladder,Esophagus,Lung,Thyroid,Tongue,Trachea"
3980,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,RVDPIGPLSY,10,3,2.85,12,5,...,"Adrenal gland,Aorta,Bone marrow,Cerebellum,Col...",4,2,6,4,1,6,"Aorta,Bone marrow,Cerebellum,Colon",Thymus,"Adrenal gland,Esophagus,Gallbladder,Lung,Thyro..."
3981,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,NVDPVQHTY,9,4,2.40,35,7,...,"Adrenal gland,Aorta,Bladder,Bone marrow,Brain,...",15,4,16,9,3,10,"Aorta,Bone marrow,Brain,Colon,Heart,Kidney,Liv...","Spleen,Testis,Thymus","Adrenal gland,Bladder,Esophagus,Gallbladder,Lu..."
3982,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,EIDKNDHLY,9,4,2.45,12,6,...,"Adrenal gland,Colon,Gallbladder,Liver,Lung,Tes...",2,3,7,2,2,6,"Colon,Liver","Testis,Thymus","Adrenal gland,Gallbladder,Lung,Thyroid,Tongue,..."
3983,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,VSDPVGVLY,9,4,2.65,7,4,...,"Bone marrow,Colon,Lung,Thymus,Thyroid,Trachea",2,2,3,2,1,3,"Bone marrow,Colon",Thymus,"Lung,Thyroid,Trachea"
3984,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,YTDPVGVLY,9,4,2.65,1,1,...,Liver,1,0,0,1,0,0,Liver,NaN,NaN
3985,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,VVDNLQHLY,9,4,2.70,5,4,...,"Lung,Thymus,Thyroid,Tongue",0,2,3,0,1,3,NaN,Thymus,"Lung,Thyroid,Tongue"
3986,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,VLDFITHLY,9,4,2.70,2,2,...,Thymus,0,2,0,0,1,0,NaN,Thymus,NaN
3987,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,EVAPAGASY,9,4,2.80,3,2,...,"Esophagus,Spleen,Thymus",0,2,1,0,2,1,NaN,"Spleen,Thymus",Esophagus


In [94]:
df_concat = pd.concat([df_mage_evd, df_mage_a3_kva, df_prame])

In [95]:
df_concat

,job,query_epitope,query_allele,label,peptide,pep_len,unit_dist,weighted_dist,n_hits,n_donors,...,tissues,protected_hits,tolerated_hits,uncertain_hits,protected_tissues_n,tolerated_tissues_n,uncertain_tissues_n,protected_tissues,tolerated_tissues,uncertain_tissues
3978,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,EVLDFITHLY,10,3,2.70,1,1,...,Thymus,0,1,0,0,1,0,NaN,Thymus,NaN
3979,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,YVDSEGHLY,9,3,2.75,13,4,...,"Aorta,Bladder,Colon,Esophagus,Heart,Lung,Testi...",3,3,7,3,2,6,"Aorta,Colon,Heart","Testis,Thymus","Bladder,Esophagus,Lung,Thyroid,Tongue,Trachea"
3980,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,RVDPIGPLSY,10,3,2.85,12,5,...,"Adrenal gland,Aorta,Bone marrow,Cerebellum,Col...",4,2,6,4,1,6,"Aorta,Bone marrow,Cerebellum,Colon",Thymus,"Adrenal gland,Esophagus,Gallbladder,Lung,Thyro..."
3981,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,NVDPVQHTY,9,4,2.40,35,7,...,"Adrenal gland,Aorta,Bladder,Bone marrow,Brain,...",15,4,16,9,3,10,"Aorta,Bone marrow,Brain,Colon,Heart,Kidney,Liv...","Spleen,Testis,Thymus","Adrenal gland,Bladder,Esophagus,Gallbladder,Lu..."
3982,MAGEA3_A01_EVDPIGHLY_on,EVDPIGHLY,A*01:01,weighted,EIDKNDHLY,9,4,2.45,12,6,...,"Adrenal gland,Colon,Gallbladder,Liver,Lung,Tes...",2,3,7,2,2,6,"Colon,Liver","Testis,Thymus","Adrenal gland,Gallbladder,Lung,Thyroid,Tongue,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1889,PRAME_A02_SLLQHLIGL_on,SLLQHLIGL,A*02:01,weighted,YQLQHLTL,8,4,3.10,1,1,...,Bone marrow,1,0,0,1,0,0,Bone marrow,NaN,NaN
1890,PRAME_A02_SLLQHLIGL_on,SLLQHLIGL,A*02:01,weighted,VLQQHLETL,9,4,3.20,1,1,...,Ovary,0,1,0,0,1,0,NaN,Ovary,NaN
1891,PRAME_A02_SLLQHLIGL_on,SLLQHLIGL,A*02:01,weighted,SPTVHLGL,8,4,3.25,1,1,...,Bone marrow,1,0,0,1,0,0,Bone marrow,NaN,NaN
1892,PRAME_A02_SLLQHLIGL_on,SLLQHLIGL,A*02:01,weighted,RLLESLIRL,9,4,3.25,1,1,...,Ovary,0,1,0,0,1,0,NaN,Ovary,NaN


In [96]:
df_concat.to_csv("the-list.csv")

In [97]:
with open("peptides.txt", "w") as f:
    for p in df_concat.peptide:
        f.write("%s\n" % p)
    f.write("KMAELVHFL\n")
    

In [98]:
!cat peptides.txt

EVLDFITHLY
YVDSEGHLY
RVDPIGPLSY
NVDPVQHTY
EIDKNDHLY
VSDPVGVLY
YTDPVGVLY
VVDNLQHLY
VLDFITHLY
EVAPAGASY
EEVDVPIKLY
ESDPIVAQY
KVADLVLML
KVMELLVHL
RLAEVVHEL
SLAELVHAV
KLAEFIDFL
KVLDFEHFL
KVVALVHAV
NVAELLHQV
KVASLLHQV
SIAEVVHQL
LVADLVHTV
YVADLVHSA
MVAEILHHL
KLAEGVQLL
KMAEMLVEL
KLAELEEAL
KVDEIVVL
KVQEQVHKV
KVDEAVAVL
SVASLVIFV
GLAELGHRL
EVAEFVQSL
VLAELVAKL
QVAERVEAL
VVAVLPHIL
EVAELFQRL
PYAERVYFL
SLLQHLIGL
SLLQHVLLL
NLLQALITL
SLLQTNIAL
SLLKNLIYL
ELLQHHGL
SLLDKIIGA
SLLSELLGV
SLMSHAIEL
SLLDSVVGL
SLLEKSLGL
SILQTLILV
SLLLKLIAV
SLLQSILQL
SLSQELVGV
SLIEVVIGL
SLLDNMIGV
ELLQKVITL
ILIEHLYGL
SVLGALIGV
SLLQRSMSL
SLLSGLVRL
SLVPHNYGL
SLAQYLINV
SLLSHVEQL
SILSLLIKL
SLLSDIIAL
SLAQYLINA
SLFPHAICL
SELQVLLGY
FLLQHVQEL
SLFEHFIEL
VLQQRLIAL
VLLSHLSYL
SLIGHLQTL
KLLKNLIGI
SLLDTLVLL
SLLPQLIEV
SILDGLIHL
SLLPQLTGA
ELLKRLIAL
SLLLEVIEL
KLLQNNYGL
ALLQRLEAL
RLLQLLLQL
SLIKHKIML
ILLSHLHYL
SLLTQPIFL
DLLQRLDAL
ALLPHLYTL
SLLDPLWTL
FLLQHQTFL
SLFQSRISL
KLYQHEINL
ALLDNNIGL
LLSHLHYL
SLLEEKILL
YQLQHLTL
VLQQHLETL
SPTVHLGL
RLLESLIRL
SL

In [99]:
len(df_concat.peptide)

101

In [100]:
len(set(df_concat.peptide))

101